# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from usl_models.atmo_ml.model import AtmoModel, AtmoModelParams
from usl_models.atmo_ml import dataset
from google.cloud import storage

# climateiq-study-area-feature-chunks/NYC_Heat/NYC_summer_2000_01p
# Define bucket names and folder paths
data_bucket_name = "climateiq-study-area-feature-chunks"
label_bucket_name = "climateiq-study-area-label-chunks"
sim_names = ["NYC_Heat_Test/NYC_summer_2000_01p"]
time_steps_per_day = 6
batch_size = 2

2024-12-02 19:33:12.864102: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 19:33:12.914061: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 19:33:12.914096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 19:33:12.915302: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 19:33:12.923184: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Create the training dataset using create_atmo_dataset with sim_names
client = storage.Client(project="climateiq")
ds = dataset.load_dataset(
    data_bucket_name=data_bucket_name,
    label_bucket_name=label_bucket_name,
    sim_names=sim_names,
    timesteps_per_day=time_steps_per_day,
    max_blobs=30,
)
ds = ds.batch(batch_size=4)
train_dataset, val_dataset, test_dataset = dataset.split_dataset(
    ds, train_frac=0.7, val_frac=0.15, test_frac=0.15
)

2024-12-02 19:33:15.579561: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38364 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0
2024-12-02 19:33:27.845595: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (1, 200, 200, 22) where an element of shape (200, 200, 22) was expected.
Traceback (most recent call last):

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

Type

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_4_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element of shape (1, 200, 200, 22) where an element of shape (200, 200, 22) was expected.
Traceback (most recent call last):

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/opt/conda/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (1, 200, 200, 22) where an element of shape (200, 200, 22) was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext] name: 

In [ ]:
# Initialize the Atmo Model
model_params = AtmoModelParams()
model = AtmoModel(model_params)
# model._model.summary()

In [ ]:
# Train the model
model.fit(train_dataset, epochs=1)

lu_index_input (None, 200, 200)
lu_index_embedded_flat Tensor("atmo_conv_lstm_10/embedding_10/embedding_lookup/Identity:0", shape=(None, 40000, 8), dtype=float32)
lu_index_embedded (None, 200, 200, 8)
spatial_cnn_output (None, 6, 50, 50, 128)
lstm_output (None, 5, 50, 50, 512)
permutedshape (None, 50, 50, 5, 512)
trconv_input (None, 8, 50, 50, 256)
outputs[0] (None, 8, 200, 200, 1)
lu_index_input (None, 200, 200)
lu_index_embedded_flat Tensor("atmo_conv_lstm_10/embedding_10/embedding_lookup/Identity:0", shape=(None, 40000, 8), dtype=float32)
lu_index_embedded (None, 200, 200, 8)
spatial_cnn_output (None, 6, 50, 50, 128)
lstm_output (None, 5, 50, 50, 512)
permutedshape (None, 50, 50, 5, 512)
trconv_input (None, 8, 50, 50, 256)
outputs[0] (None, 8, 200, 200, 1)


2024-11-21 20:19:31.242676: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inatmo_conv_lstm_10/conv_lstm/conv_lstm2d_10/while/body/_1/atmo_conv_lstm_10/conv_lstm/conv_lstm2d_10/while/dropout_7/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-11-21 20:19:32.134690: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 16365553684496521654
2024-11-21 20:19:32.134736: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 7210658781760474270
2024-11-21 20:19:32.134742: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 15804210494231023062
2024-11-21 20:19:32.134747: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 2249478919593754292


InvalidArgumentError: Graph execution error:

Detected at node atmo_conv_lstm_10/Reshape_2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/opt/conda/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/conda/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/conda/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/conda/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/conda/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/opt/conda/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/opt/conda/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/opt/conda/lib/python3.11/site-packages/ipykernel/kernelbase.py", line 545, in dispatch_queue

  File "/opt/conda/lib/p

In [ ]:
inputs, labels = next(iter(train_dataset))
{key: tensor.shape for key, tensor in inputs.items()}

{'spatiotemporal': TensorShape([2, 6, 200, 200, 12]),
 'spatial': TensorShape([2, 200, 200, 22]),
 'lu_index': TensorShape([2, 200, 200])}

In [ ]:
model._model.summary()

Model: "atmo_conv_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  488       
                                                                 
 spatial_cnn (Sequential)    (None, 50, 50, 128)       252992    
                                                                 
 spatiotemporal_cnn (Sequen  (None, None, 50, 50, 64   30480     
 tial)                       )                                   
                                                                 
 conv_lstm (Sequential)      (None, None, 50, 50, 51   45877248  
                             2)                                  
                                                                 
 t2_output_cnn (Sequential)  (None, None, 200, 200,    69729     
                             1)                                  
                                                    

In [ ]:
# Test calling the model on some training data
inputs, labels = next(iter(train_dataset))
prediction = model.call(inputs)
print("Prediction shape:", prediction.shape)

lu_index_input (2, 200, 200)
lu_index_embedded_flat tf.Tensor(
[[[-0.0290212  -0.00429965  0.0070921  ...  0.04053745 -0.02890884
    0.00442706]
  [ 0.04117144  0.03346105  0.00490087 ...  0.01888017  0.02810446
    0.0290962 ]
  [ 0.01980658  0.01989866 -0.02752843 ...  0.02466523  0.03207915
    0.01597965]
  ...
  [ 0.04117144  0.03346105  0.00490087 ...  0.01888017  0.02810446
    0.0290962 ]
  [-0.00921012  0.02187858 -0.02844076 ...  0.04788701  0.01579146
   -0.04698772]
  [-0.00921012  0.02187858 -0.02844076 ...  0.04788701  0.01579146
   -0.04698772]]

 [[-0.0290212  -0.00429965  0.0070921  ...  0.04053745 -0.02890884
    0.00442706]
  [ 0.04117144  0.03346105  0.00490087 ...  0.01888017  0.02810446
    0.0290962 ]
  [ 0.01980658  0.01989866 -0.02752843 ...  0.02466523  0.03207915
    0.01597965]
  ...
  [ 0.04117144  0.03346105  0.00490087 ...  0.01888017  0.02810446
    0.0290962 ]
  [-0.00921012  0.02187858 -0.02844076 ...  0.04788701  0.01579146
   -0.04698772]
  [-0.0092